# Gemini API: Prompting with Video

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook provides a quick example of how to prompt Gemini 1.5 Pro using a video file. In this case, you'll use a short clip of [Big Buck Bunny](https://peach.blender.org/about/).

In [2]:
# !pip install -U -q google-generativeai

In [3]:
import google.generativeai as genai

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [4]:
# from google.colab import userdata
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# genai.configure(api_key=GOOGLE_API_KEY)

import os
os.environ["GEMINI_API_KEY"]= "INSERT KEY HERE"
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))


## Extract frames

The Gemini API currently does not support video files directly. Instead, you can provide a series of timestamps and image files.

We will extract 1 frame a second from a the short film "Big Buck Bunny" file using [OpenCV](https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html).

> "Big Buck Bunny" is (c) copyright 2008, Blender Foundation / www.bigbuckbunny.org and [licensed](https://peach.blender.org/about/) under the [Creative Commons Attribution 3.0](http://creativecommons.org/licenses/by/3.0/) License.

Note: You can also [upload your own files](https://github.com/google-gemini/cookbook/tree/main/examples/Upload_files.ipynb) to use.

In [5]:
video_file_name = "mp4_converted_video.mp4"

Use OpenCV to extract image frames from the video at 1 frame per second.

In [6]:
!ls frames/mp4_converted_video_mp4_frame00  

ls: cannot access 'frames/mp4_converted_video_mp4_frame00': No such file or directory


In [7]:
import cv2
import os
import shutil

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
  print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
  create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
  vidcap = cv2.VideoCapture(video_file_path)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  frame_duration = 1 / fps  # Time interval between frames (in seconds)
  output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
  frame_count = 0
  count = 0
  while vidcap.isOpened():
      success, frame = vidcap.read()
      if not success: # End of video
          break
      if int(count / fps) == frame_count: # Extract a frame every second
          print(frame_count)
          min = frame_count // 60
          sec = frame_count % 60
          time_string = f"{min:02d}_{sec:02d}"
          image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
          output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
          print(output_filename)
          cv2.imwrite(output_filename, frame)
          frame_count += 1
      count += 1
  vidcap.release() # Release the capture object\n",
  print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

extract_frame_from_video(video_file_name)

Extracting mp4_converted_video.mp4 at 1 frame per second. This might take a bit...
0
frames\mp4_converted_video_mp4_frame00_00.jpg
1
frames\mp4_converted_video_mp4_frame00_01.jpg
2
frames\mp4_converted_video_mp4_frame00_02.jpg
3
frames\mp4_converted_video_mp4_frame00_03.jpg
4
frames\mp4_converted_video_mp4_frame00_04.jpg
5
frames\mp4_converted_video_mp4_frame00_05.jpg
6
frames\mp4_converted_video_mp4_frame00_06.jpg
7
frames\mp4_converted_video_mp4_frame00_07.jpg
8
frames\mp4_converted_video_mp4_frame00_08.jpg
9
frames\mp4_converted_video_mp4_frame00_09.jpg
10
frames\mp4_converted_video_mp4_frame00_10.jpg
11
frames\mp4_converted_video_mp4_frame00_11.jpg
12
frames\mp4_converted_video_mp4_frame00_12.jpg
13
frames\mp4_converted_video_mp4_frame00_13.jpg
14
frames\mp4_converted_video_mp4_frame00_14.jpg
15
frames\mp4_converted_video_mp4_frame00_15.jpg
16
frames\mp4_converted_video_mp4_frame00_16.jpg
17
frames\mp4_converted_video_mp4_frame00_17.jpg
18
frames\mp4_converted_video_mp4_frame00_18.

## Upload frames using the File API

Once we have the frames extracted, we are ready to upload the frames to the API.

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API.

We will just upload 10 frames so this example runs quickly. You can modify the code below to upload the entire video.

In [9]:
import os

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
# Only upload a 10 second slice of files to reduce upload time.
# Change full_video to True to upload the whole video.
full_video = False

uploaded_files = []
print(f'Uploading {len(files_to_upload) if full_video} files. This might take a bit...')

for file in files_to_upload if full_video else files_to_upload:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

Uploading 10 files. This might take a bit...
Uploading: frames\mp4_converted_video_mp4_frame00_00.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_01.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_02.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_03.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_04.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_05.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_06.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_07.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_08.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_09.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_10.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_11.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_12.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_13.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_14.jpg...
Uploading: frames\mp4_converted_video_mp4_frame00_15.jp

## List Files

After uploading the file, you can verify the API has successfully received the files by calling `files.list`.

`files.list` lets you see all files that have been uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique.

In [ ]:
# List files uploaded in the API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

https://generativelanguage.googleapis.com/v1beta/files/yxy6h76dqllw
https://generativelanguage.googleapis.com/v1beta/files/tae9e09r72ll
https://generativelanguage.googleapis.com/v1beta/files/uz398cjo7t60
https://generativelanguage.googleapis.com/v1beta/files/yqzehq5lvhwj
https://generativelanguage.googleapis.com/v1beta/files/mc317onbtu0a
https://generativelanguage.googleapis.com/v1beta/files/jwttpvedfrm4
https://generativelanguage.googleapis.com/v1beta/files/5pgnm3bnbpfz
https://generativelanguage.googleapis.com/v1beta/files/6dhg3it8zrtq
https://generativelanguage.googleapis.com/v1beta/files/9krlw49hf7eb
https://generativelanguage.googleapis.com/v1beta/files/p2zw4163464x
https://generativelanguage.googleapis.com/v1beta/files/b8cuc8ko76oq
https://generativelanguage.googleapis.com/v1beta/files/960soh6ws5z4
https://generativelanguage.googleapis.com/v1beta/files/7gidca7npcjj
https://generativelanguage.googleapis.com/v1beta/files/mn4zetnxjqs0
https://generativelanguage.googleapis.com/v1beta

## Generate Content

After the file has been uploaded, you can make `GenerateContent` requests that reference the File API URI.

To understand videos with Gemini 1.5 Pro, provide 2 consecutive `Part`s for each frame: a `text` part with the **timestamp** and `fileData` part with the frame's **image URI**:

```
part { text = "00:00" }
part { fileData = fileData {
  fileUri = "https://generativelanguage.googleapis.com/v1/files/frame-0"
  mimeType = "image/jpeg"
}}
```

In [ ]:
file.timestamp

'00_36'

In [13]:
# Create the prompt.
prompt = "Below you are provided the frames of a video with their corresponding timestamps below each frame. If a crime was commited, identify the earliest frame where it was commited and the timestamp of the frame and explain why you selected that frame. If no crime was commited, write 'No crime commited"

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp.replace("_", ":"))
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

ValueError: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.

## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
for file in uploaded_files:
  genai.delete_file(file.response.name)
  print(f'Deleted {file.file_path} at URI {file.response.uri}')
print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

Deleting 10 images. This might take a bit...
files/ax2qg09qlt8z
Deleted /content/frames/BigBuckBunny_320x180_mp4_frame00:43.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/ax2qg09qlt8z
files/wt6zcf2sr5f6
Deleted /content/frames/BigBuckBunny_320x180_mp4_frame00:44.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/wt6zcf2sr5f6
files/f1hh0z86bg72
Deleted /content/frames/BigBuckBunny_320x180_mp4_frame00:45.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/f1hh0z86bg72
files/5gmr6rkakb13
Deleted /content/frames/BigBuckBunny_320x180_mp4_frame00:46.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/5gmr6rkakb13
files/icbgp7xsxhe3
Deleted /content/frames/BigBuckBunny_320x180_mp4_frame00:47.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/icbgp7xsxhe3
files/iilxn5wjze28
Deleted /content/frames/BigBuckBunny_320x180_mp4_frame00:48.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/iilxn5wjze28
files/q

## Learning more

The File API lets you upload a variety of multimodal MIME types, including images and audio formats. The File API handles inputs that can be used to generate content with [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent).

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API.

* Learn more about the [File API](https://github.com/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb) with the quickstart.

* Learn more about prompting with [media files](https://ai.google.dev/tutorials/prompting_with_media) in the docs, including the supported formats and maximum length.